# ONNX Tutorial - Super Resolution Vision Model - End to End

The tutorial demonstrates complete lifecycle of neural model including  
- training (or rather exporting pretrained weights)
- storage (this is where BlobHub comes into play)
- and subsequent inference.

The model used in this tutorial is trained to scale up images (increase image resolution).

References: 
- Original tutorial  
  https://pytorch.org/tutorials/advanced/super_resolution_with_onnxruntime.html

## Table of Contents

- [Prerequisites](#Prerequisites)
- [Install Dependencies](#Install-Dependencies) 
- [Export the Model](#Export-the-Model)  
- [Upload Model to BlobHub](#Upload-Model-to-BlobHub)  
- [Download Model from BlobHub](#Download-Model-from-BlobHub)  
- [Run Model Inference](#Run-Model-Inference)  
- [Compare Results](#Compare-Results)  

## Prerequisites

It is required to create BlobHub blob and API key (with `write` access level) in order to execute the tutorial. 

In [ ]:
ORG_ID = "<< Org ID >>"
BLOB_ID = "<< Blob ID >>"
API_KEY = "<< API Key >>"

## Install Dependencies

In [ ]:
!pip install torch

In [ ]:
!pip install torchvision

In [ ]:
!pip install blobhub

In [ ]:
!pip install onnx

In [ ]:
!pip install onnxruntime

In [ ]:
!pip install Pillow

In [ ]:
!pip install requests

## Export the Model 

Define model topology:

In [ ]:
import torch.nn as nn
import torch.nn.init as init

class SuperResolutionNet(nn.Module):
    def __init__(self, upscale_factor, inplace=False):
        super(SuperResolutionNet, self).__init__()

        self.relu = nn.ReLU(inplace=inplace)
        self.conv1 = nn.Conv2d(1, 64, (5, 5), (1, 1), (2, 2))
        self.conv2 = nn.Conv2d(64, 64, (3, 3), (1, 1), (1, 1))
        self.conv3 = nn.Conv2d(64, 32, (3, 3), (1, 1), (1, 1))
        self.conv4 = nn.Conv2d(32, upscale_factor ** 2, (3, 3), (1, 1), (1, 1))
        self.pixel_shuffle = nn.PixelShuffle(upscale_factor)

        self._initialize_weights()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = self.pixel_shuffle(self.conv4(x))
        return x

    def _initialize_weights(self):
        init.orthogonal_(self.conv1.weight, init.calculate_gain('relu'))
        init.orthogonal_(self.conv2.weight, init.calculate_gain('relu'))
        init.orthogonal_(self.conv3.weight, init.calculate_gain('relu'))
        init.orthogonal_(self.conv4.weight)

# Create the super-resolution model by using the above model definition.
torch_model = SuperResolutionNet(upscale_factor=3)

Load previously trained model parameters:

In [ ]:
import torch
import torch.utils.model_zoo as model_zoo

# Load pretrained model weights
model_url = 'https://s3.amazonaws.com/pytorch/test_data/export/superres_epoch100-44c6958e.pth'
batch_size = 1

# Initialize model with the pretrained weights
map_location = lambda storage, loc: storage
if torch.cuda.is_available():
    map_location = None
torch_model.load_state_dict(model_zoo.load_url(model_url, map_location=map_location))

# set the model to inference mode
torch_model.eval()

Export model into ONNX format:

In [ ]:
onnx_file_path = "super_resolution.onnx"

# Input to the model
torch_input = torch.randn(batch_size, 1, 224, 224, requires_grad=True)
torch_out = torch_model(torch_input)

# Export the model
torch.onnx.export(
    torch_model,               # model being run
    torch_input,               # model input (or a tuple for multiple inputs)
    onnx_file_path,            # where to save the model (can be a file or file-like object)
    export_params=True,        # store the trained parameter weights inside the model file
    opset_version=10,          # the ONNX version to export the model to
    do_constant_folding=True,  # whether to execute constant folding for optimization
    input_names = ['input'],   # the model's input names
    output_names = ['output'], # the model's output names
    dynamic_axes={'input' : {0 : 'batch_size'},    # variable lenght axes
                'output' : {0 : 'batch_size'}})

## Upload Model to BlobHub

Model uploading code:

In [ ]:
from blobhub.blob import Blob, Revision
from blobhub.presets.onnx import Onnx, Model        
        
# Find blob
blob = Blob(org_id=ORG_ID, blob_id=BLOB_ID, api_key=API_KEY)
revision = blob.revisions.latest()

# Revision must be "draft" and "ready"
assert revision[Revision.FIELD_PHASE] == Revision.PHASE_DRAFT
assert revision[Revision.FIELD_STATUS] == Revision.STATUS_READY

# Initialize preset
onnx = Onnx(revision=revision)

# Upload the model to the revision
initial_model = Model.from_local_file(onnx=onnx, path=onnx_file_path)
success = onnx.upload(model=initial_model)
assert True == success

## Download Model from BlobHub

Model downloading code:

In [ ]:
from blobhub.blob import Blob, Revision
from blobhub.presets.onnx import Onnx, Model       

# Find blob
blob = Blob(org_id=ORG_ID, blob_id=BLOB_ID, api_key=API_KEY)
revision = blob.revisions.latest()

# Initialize preset
onnx = Onnx(revision=revision)

# Download and save the model
downloaded_model = onnx.download()
assert None != downloaded_model

Downloaded model is stored locally and is accessible under:

In [ ]:
downloaded_model.path

## Run Model Inference

Check model correctness (ONNX model consistency check):

In [ ]:
import onnx

onnx_model = onnx.load(downloaded_model.path)
onnx.checker.check_model(onnx_model)

Initialize ONNX runtime:

In [ ]:
import onnxruntime

ort_session = onnxruntime.InferenceSession(downloaded_model.path)

Load input image and convert to tensor:

In [ ]:
import requests
from io import BytesIO
from PIL import Image
import torchvision.transforms as transforms

# Load the image 
input_image_url = "https://static.blobhub.io/revisions/01cba831-d693-4adf-b0c8-ded7e553c019/bridge_224x224.jpg"
response = requests.get(input_image_url)
input_image_original = Image.open(BytesIO(response.content))

# Resize image to match model input dimensions
resize = transforms.Resize([224, 224])
input_image = resize(input_image_original)

# Convert image into YCbCr color space 
img_ycbcr = input_image.convert("YCbCr")
img_in_y, img_in_cb, img_in_cr = img_ycbcr.split()

# Create tensor based on luma signal
to_tensor = transforms.ToTensor()
img_in_y = to_tensor(img_in_y)
img_in_y.unsqueeze_(0)

Run inference:

In [ ]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [ ]:
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(img_in_y)}
ort_outs = ort_session.run(None, ort_inputs)
img_out_y_out = ort_outs[0]

Construct image object based on the model output:

In [ ]:
import numpy as np

# Convert model output into an image (just Y channel)
img_out_y = Image.fromarray(np.uint8((img_out_y_out[0] * 255.0).clip(0, 255)[0]), mode='L')

# Reconstruct the image by using model-resized luma channel and maually-resized chroma components
output_image = Image.merge(
    "YCbCr", [
        img_out_y,
        img_in_cb.resize(img_out_y.size, Image.BICUBIC),
        img_in_cr.resize(img_out_y.size, Image.BICUBIC),
    ]).convert("RGB")

## Compare Results

Original image:

In [ ]:
input_image

Resized image:

In [ ]:
output_image